# Consistent Bayes: Harmonic Oscillator Example
---

Copyright 2018 Michael Pilosov


### Import Libraries


In [1]:
from matplotlib import pyplot as plt
# %matplotlib inline
# %matplotlib notebook
plt.rcParams.update({'font.size': 14})
plt.rcParams['figure.figsize'] = 8, 5
import numpy as np
from cbayes import sample, solve, distributions

import ipyparallel as ipp # import the client
rc = ipp.Client()
dv = rc[:]
rc.ids # list the ids of the engine the client can communicate with


/home/mpilosov/anaconda3/envs/test_py_env/lib/python3.6/site-packages/ipyparallel/client/client.py:458: RuntimeWarning: 
            Controller appears to be listening on localhost, but not on this machine.
            If this is true, you should specify Client(...,sshserver='you@math-ws-204')
            or instruct your controller to listen on an external IP.
  RuntimeWarning)


[0, 1, 2, 3]

In [2]:
%%px

lambda_ref = np.array([[0, 1]])

def MSE_generator(model, obs_data, sigma=1):   # this generates a sum of squared residuals.
    def QoI_fun(inputs): # that conforms to our desired model input
        M = len(obs_data)
        predictions = model(inputs)
        assert predictions.shape[1] == M
        residuals = predictions - obs_data
        QoI = (1./M)*np.sum( (residuals/sigma)**2, axis=1 )
        return QoI
    return QoI_fun

# The model of the harmonic oscillator 
def makemodel(t):
    def model(lam = lambda_ref ):
#         return np.cos(lam[:,1]*t + np.arccos(lam[:,0]) )
        QoI = np.cos(np.outer(lam[:,1],t) + np.arccos(lam[:,0]).reshape(len(lam[:,0]),1) )
        if QoI.shape[0] == 1:
            return QoI.ravel() # this allows support for simpler 1D plotting.
        else:
            return QoI
    return model


In [3]:
num_samples = np.linspace(5E3, 25E3, 5)
num_obs = np.linspace(5,25,5)
true_observational_variance = 0.1/np.linspace(4,6,3)
perceived_observational_variance = 0.1/np.linspace(4,6,5)
window_length = np.array([2,4,6,8,10])
S = np.array([ num_obs.astype('int'), num_samples, true_observational_variance, perceived_observational_variance, window_length])
num_iterations = np.prod([ S[i].shape[0] for i in range(S.shape[0]) ])
SS = np.array( np.meshgrid(*S) )
trial_vector = SS.ravel().reshape(S.shape[0],num_iterations).T
print(trial_vector[0:5,:])

[[5.0e+00 5.0e+03 2.5e-02 2.5e-02 2.0e+00]
 [5.0e+00 5.0e+03 2.5e-02 2.5e-02 4.0e+00]
 [5.0e+00 5.0e+03 2.5e-02 2.5e-02 6.0e+00]
 [5.0e+00 5.0e+03 2.5e-02 2.5e-02 8.0e+00]
 [5.0e+00 5.0e+03 2.5e-02 2.5e-02 1.0e+01]]


# Example Summary

---

The second example is defined by the model for a harmonic oscillator given by the initial value problem  

\begin{equation}
	\begin{cases}
		\frac{d^2u}{dt^2} &= -\lambda_2^2 u, \ t>0, \\
		u(0) &= \lambda_1. 
	\end{cases}
\end{equation}

Choosing a particular parameter $\lambda=(\lambda_1,\lambda_2)\in\Lambda\subset[-1,1]\times\mathbb{R}^+$ corresponds to fixing an initial condition, $\lambda_1\in[-1,1]$, and frequency of oscillation, $\lambda_2$, where we again assume that $\lambda_2>0$.

The solution to the above is
\begin{equation}
	u(t) = \cos(\lambda_2 t + \arccos(\lambda_1)).
\end{equation}

---
## Define Functions

In [5]:
%%px
mins = np.array([-0.005, 0.9])
maxs = np.array([.005, 1.1])
nbins = 100
X = grid(nbins, mins, maxs)

def run_trial(trial):
    num_observations, num_samples, sd_true, sd_guess, max_time = trial
    num_observations = int(num_observations)
    num_samples = int(num_samples)
    
    t = np.linspace(0, max_time, num_observations)
    model = makemodel(t)
    u_true = model()
    obs_data = u_true + np.random.randn(int(num_observations))*sd_true
    PtO_fun = MSE_generator(model, obs_data, sd_guess)
    
    s_set = sample.sample_set(size=(num_samples, 2))
    s_set.set_dist('beta', kwds={'a': 1, 'b': 1, 'loc': -0.25, 'scale': 0.5}, dim=0) # initial condition
    s_set.set_dist('beta', kwds={'a': 1, 'b': 1, 'loc': .5, 'scale': 1}, dim=1) # frequency
    s_set.generate_samples()
    p_set = sample.map_samples_and_create_problem(s_set, PtO_fun)
    p_set.compute_pushforward_dist()
    p_set.set_observed_dist('gamma', {'a':num_observations/2, 'scale':2/num_observations}, dim=0)
    p_set.set_ratio()
    #     solve.problem(p_set) # if you care to perform accept/reject on the prior, uncomment this.
#     accept_inds = p_set.accept_inds
#     lam_accept = p_set.input.samples[accept_inds,:]
#     num_accept = len(accept_inds)
#     print('Number accepted: %d = %2.2f%%'%(num_accept, 100*np.float(num_accept)/num_samples))
    
    # Perform accept/reject on that grid you see plotted above.
    Y = PtO_fun(X)
    inds = solve.perform_accept_reject(X, p_set.compute_ratio(Y))
    abs_error = [*np.abs( lambda_ref[0,:] - np.mean(X[inds,:],axis=0) )]
    rel_error = [*np.abs( p_set.observed_dist.mean()-np.mean(Y[inds],axis=0))/p_set.observed_dist.mean() , *np.abs( p_set.observed_dist.std()-np.std(Y[inds],axis=0))/p_set.observed_dist.std()]
#     print('Abs Error in Parameter and MAP:', abs_error )
#     print('Param. Test Stats for Gamma Dist:', rel_error)
#     print(' ')
    

    post_eval = p_set.evaluate_posterior(X)
    return {'p': post_eval, 'a': abs_error, 'r': rel_error}


In [6]:
dv.scatter('trial_chunk', trial_vector[0:5,:])

<AsyncResult: scatter>

In [10]:
%%capture output
%%px
D = list(map(run_trial, trial_chunk))

In [11]:
output.show()

In [12]:
Dg = np.array(list(dv.gather('D').get() ))
np.save("savefile_uni_hd.npy", ell_1_error)

NameError: name 'L1_error' is not defined

In [ ]:
# Make the plot
# xi = X[:,0].reshape(nbins, nbins)
# yi = X[:,1].reshape(nbins, nbins)
# zi = post_eval.reshape(nbins, nbins)
# plt.pcolormesh(xi, yi, zi)
# plt.scatter(lam_true[0,0], lam_true[0,1], c='white', edgecolor='black', s=50)
# plt.show()
